In [ ]:
import pandas as pd

def process(df):
    df['date'] = pd.to_datetime(df['timestamp'], unit='s')

    start_of_february = pd.to_datetime("2024-02-01")
    end_of_february = pd.to_datetime("2024-02-29")
    start_of_march = pd.to_datetime("2024-03-01")


    df['end_date'] = df['date'] + pd.to_timedelta(df['billing_period']-1, unit='D')

    active_in_february = df[(df['date'] < end_of_february) & (df['end_date'] >= start_of_february)]

    # Рассчитываем количество дней активности подписки в феврале
    active_in_february['active_days_in_february'] = (active_in_february[['date', 'end_date']]
                                                      .apply(lambda x: min(x['end_date'], end_of_february) - max(x['date'], start_of_february), axis=1).dt.days)

    # Рассчитываем финансовый вклад для каждой подписки
    active_in_february['financial_contribution'] = active_in_february['billing_total_price_usd'] / active_in_february['active_days_in_february']

    # Группируем данные по пользователям и суммируем финансовый вклад каждого пользователя
    user_financial_contribution = active_in_february.groupby('user_id')['financial_contribution'].sum()

    # Находим ТОП-3 пользователей по финансовому вкладу
    top_3_users = user_financial_contribution.nlargest(3)

    # Считаем совокупный финансовый вклад ТОП-3 пользователей
    total_contribution_top_3 = top_3_users.sum()

    return total_contribution_top_3

df = pd.read_csv("sample_1.csv",index_col="id")
print(process(df))


15.202380952380953


In [ ]:
import pandas as pd

df = pd.read_csv("sample_1.csv", index_col="id")
df['date'] = pd.to_datetime(df['timestamp'], unit='s')
df.drop(columns=['timestamp'], inplace=True)

# Рассчитываем дату начала и конца февраля 2024 года
start_of_february = pd.to_datetime("2024-02-01")
end_of_february = pd.to_datetime("2024-02-29")  # Последний день февраля

# Рассчитываем дату окончания подписки, учитывая начальную дату и период подписки
df['end_date'] = df['date'] + pd.to_timedelta(df['billing_period'] - 1, unit='D')

# Фильтруем подписки, которые были активны в феврале 2024 года
active_in_february = df[(df['date'] < end_of_february) & (df['end_date'] >= start_of_february)]

# Рассчитываем количество дней активности подписки в феврале
active_in_february['active_days_in_february'] = (active_in_february[['date', 'end_date']]
                                                  .apply(lambda x: min(x['end_date'], end_of_february) - max(x['date'], start_of_february), axis=1).dt.days)

active_in_february


,user_id,billing_period,billing_total_price_usd,date,end_date,active_days_in_february
id,,,,,,
0,26,74,95,2024-02-16 17:21:07,2024-04-29 17:21:07,12
1,40,25,46,2024-02-10 17:21:07,2024-03-05 17:21:07,18
2,48,50,99,2024-01-11 17:21:07,2024-02-29 17:21:07,28
3,15,32,15,2024-01-05 17:21:07,2024-02-05 17:21:07,4
4,38,84,83,2024-01-23 17:21:07,2024-04-15 17:21:07,28


In [ ]:
active_in_february['financial_contribution'] = active_in_february['billing_total_price_usd'] / active_in_february['active_days_in_february']
active_in_february

,user_id,billing_period,billing_total_price_usd,date,end_date,active_days_in_february,financial_contribution
id,,,,,,,
0,26,74,95,2024-02-16 17:21:07,2024-04-29 17:21:07,12,7.916667
1,40,25,46,2024-02-10 17:21:07,2024-03-05 17:21:07,18,2.555556
2,48,50,99,2024-01-11 17:21:07,2024-02-29 17:21:07,28,3.535714
3,15,32,15,2024-01-05 17:21:07,2024-02-05 17:21:07,4,3.750000
4,38,84,83,2024-01-23 17:21:07,2024-04-15 17:21:07,28,2.964286


In [ ]:
import pandas as pd

df = pd.read_csv("sample_2.csv", index_col="id")
df['date'] = pd.to_datetime(df['timestamp'], unit='s')
df.drop(columns=['timestamp'], inplace=True)
df

,user_id,billing_period,billing_total_price_usd,date
id,,,,
23,1,6,60,2024-01-29 19:11:23


In [ ]:
import pandas as pd

def process(df):
    # Преобразование timestamp в формат даты
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

    # Отфильтровать данные за февраль 2024 года
    feb_2024_data = df[(df['timestamp'].dt.year == 2024) & (df['timestamp'].dt.month == 2)]

    # Рассчитать финансовый вклад для каждой подписки
    feb_2024_data['financial_contribution'] = feb_2024_data['billing_total_price_usd'] / feb_2024_data['billing_period']

    # Сгруппировать данные по пользователям и посчитать суммарный финансовый вклад для каждого пользователя
    user_contribution = feb_2024_data.groupby('user_id')['financial_contribution'].sum().reset_index()

    # Найти ТОП-3 пользователей по суммарному финансовому вкладу
    top_3_users = user_contribution.nlargest(3, 'financial_contribution')

    # Вернуть сумму финансового вклада ТОП-3 пользователей
    return top_3_users['financial_contribution'].sum()

# Пример использования
data = {
    'id': [0, 1, 2, 3, 4],
    'user_id': [26, 40, 48, 15, 38],
    'timestamp': [1708104067, 1707585667, 1704993667, 1704475267, 1706030467],
    'billing_period': [74, 25, 50, 32, 84],
    'billing_total_price_usd': [95, 46, 99, 15, 83]
}

df = pd.DataFrame(data)
result = process(df)
print(result)


3.123783783783784


<ipython-input-77-ffeec71f605a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_2024_data['financial_contribution'] = feb_2024_data['billing_total_price_usd'] / feb_2024_data['billing_period']


In [ ]:
import sqlite3

def query_budgets1():
    # Подключение к базе данных
    conn = sqlite3.connect('budgets.db')
    c = conn.cursor()
    # Выполнение SQL-запроса
    c.execute('''
select * from payment_date
    ''')
    # Получение результатов
    results = c.fetchall()

    # Закрытие соединения с базой данных
    conn.close()

    return results
def query_budgets2():
    # Подключение к базе данных
    conn = sqlite3.connect('budgets.db')
    c = conn.cursor()
    # Выполнение SQL-запроса
    c.execute('''
select * from advertising_companies
    ''')
    # Получение результатов
    results = c.fetchall()

    # Закрытие соединения с базой данных
    conn.close()

    return results
def query_budgets3():
    # Подключение к базе данных
    conn = sqlite3.connect('budgets.db')
    c = conn.cursor()
    # Выполнение SQL-запроса
    c.execute('''
select * from users
    ''')
    # Получение результатов
    results = c.fetchall()

    # Закрытие соединения с базой данных
    conn.close()

    return results
# Вызов функции и вывод результатов
query_budgets1()


[(1, '2023-01-10', 1, 1, 100.0),
 (2, '2023-01-12', 1, 1, 100.0),
 (3, '2023-01-15', 1, 1, 100.0),
 (4, '2023-02-20', 1, 1, 100.0),
 (5, '2023-02-20', 1, 1, 100.0),
 (6, '2023-02-20', 1, 3, 180.0),
 (7, '2023-02-20', 2, 2, 150.0),
 (8, '2023-02-20', 2, 2, -150.0),
 (9, '2023-03-20', 1, 1, 150.0),
 (10, '2023-03-25', 1, 3, 220.0),
 (11, '2023-03-25', 3, 13, 200.0),
 (12, '2023-04-15', 1, 1, 200.0),
 (13, '2023-04-15', 1, 3, 105.0),
 (14, '2023-04-15', 4, 4, 120.0),
 (15, '2023-04-20', 1, 1, 100.0),
 (16, '2023-05-10', 1, 1, 110.0),
 (17, '2023-05-15', 1, 3, -500.0),
 (18, '2023-05-10', 5, 5, 180.0),
 (19, '2023-05-11', 1, 3, 120.0),
 (20, '2023-05-30', 6, 6, 220.0),
 (21, '2023-07-20', 7, 7, 140.0),
 (22, '2023-08-15', 12, 8, 160.0),
 (23, '2023-08-20', 12, 9, 250.0),
 (24, '2023-09-25', 1, 1, 50.0),
 (25, '2023-09-30', 12, 10, 190.0),
 (26, '2023-10-20', 1, 1, 25.0),
 (27, '2023-10-30', 12, 12, 120.0),
 (28, '2023-10-31', 13, 11, 130.0)]

In [ ]:
query_budgets2()

[(1, '2023-01-01', 0),
 (2, '2023-02-15', 1),
 (3, '2023-03-20', 1),
 (4, '2023-04-10', 1),
 (5, '2023-05-05', 1),
 (6, '2023-06-20', 1),
 (7, '2023-07-15', 1),
 (8, '2023-08-10', 0),
 (9, '2023-08-10', 0),
 (10, '2023-09-25', 1),
 (11, '2023-10-29', 1),
 (12, '2023-10-29', 0),
 (13, '2023-10-30', 1)]

In [ ]:
query_budgets3()

[(1, 'Москва', ' 2023-01-01', 8, 'Иванов'),
 (2, 'Санкт-Петербург', ' 2023-02-15', 9, 'Петров'),
 (3, 'Владимир', ' 2023-03-20', 10, 'Сидоров'),
 (4, 'Минск', ' 2023-04-10', 11, 'Козлов'),
 (5, 'Самара', ' 2023-05-10', 11, 'Крышнев'),
 (6, 'Тольятти', ' 2023-05-30', 8, 'Роверов'),
 (7, 'Москва', ' 2023-06-01', 8, 'Лузанов'),
 (8, 'Москва', ' 2023-06-15', None, 'ACME Corporation'),
 (9, 'Париж', ' 2023-06-20', None, 'Tech Solutions'),
 (10, 'Берлин', ' 2023-07-15', None, 'Global Industries'),
 (11, 'Рим', ' 2023-08-10', None, 'Innovate Co.'),
 (12, 'Токио', ' 2023-09-25', None, 'Ямамото'),
 (13, 'Пекин', ' 2023-10-30', None, 'Ли'),
 (14, 'Казань', ' 2023-11-02', 999, 'Потеряшкин')]

In [ ]:
import sqlite3

def query_budgets():
    # Подключение к базе данных
    conn = sqlite3.connect('budgets.db')
    c = conn.cursor()
    # Выполнение SQL-запроса на обновление
    c.execute('''
    UPDATE users
    SET owner_id = id
    WHERE owner_id IS NULL or owner_id NOT IN (SELECT id FROM users WHERE id IS NOT NULL)
    ''')

    # Выполнение SQL-запроса на выборку
    c.execute('''
    WITH smt AS (
    SELECT
        u.id,
        u.owner_id,
        SUM(CASE WHEN pd.sum > 0 THEN pd.sum END) AS total_sum,
        ac.create_date,
        ac.is_main
    FROM
        users u
    LEFT JOIN
        payment_date pd ON pd.user_id = u.id
    LEFT JOIN
        advertising_companies ac ON pd.rk_id = ac.id
    GROUP BY
        u.id, ac.id
),
names AS (
    SELECT
        id,
        name
    FROM
        users
)

SELECT
    n.name,
    SUM(s.total_sum) AS total_sum,
    MIN(s.create_date) AS min_create_date,
    MAX(s.is_main) AS max_is_main
FROM
    smt s
LEFT JOIN
    names n ON s.owner_id = n.id
WHERE
    s.create_date = (SELECT MIN(create_date) FROM smt) OR s.is_main = 1
GROUP BY
    n.name;
order by n.name
''')




    # Получение результатов
    results = c.fetchall()

    # Закрытие соединения с базой данных
    conn.close()

    return results

# Вызов функции и вывод результатов
results = query_budgets()
for row in results:
    print(row)


('ACME Corporation', 2120.0, '2023-01-01', 1)
('Global Industries', 200.0, '2023-10-30', 1)
('Innovate Co.', 300.0, '2023-04-10', 1)
('Tech Solutions', 150.0, '2023-02-15', 1)
('Ли', 130.0, '2023-10-29', 1)
('Ямамото', 190.0, '2023-09-25', 1)
